### Проверка того, что наше api работает

Перед началом необходимо собрать docker-образ и запустить контейнер

1. Следовать инструкциям https://github.com/Foreman1980/Docker_FlaskProject/blob/main/README.md
2. Далее уже запускать код ниже

Blood Transfusion Service Center Data Set (https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center)

Data Set Information:

To demonstrate the RFMTC marketing model (a modified version of RFM), this study 
adopted the donor database of Blood Transfusion Service Center in Hsin-Chu City 
in Taiwan. The center passes their blood transfusion service bus to one 
university in Hsin-Chu City to gather blood donated about every three months. To 
build a FRMTC model, we selected 748 donors at random from the donor database. 
These 748 donor data, each one included R (Recency - months since last 
donation), F (Frequency - total number of donation), M (Monetary - total blood 
donated in c.c.), T (Time - months since first donation), and a binary variable 
representing whether he/she donated blood in March 2007 (1 stand for donating 
blood; 0 stands for not donating blood).

Attribute Information:

Given is the variable name, variable type, the measurement unit and a brief 
description. The "Blood Transfusion Service Center" is a classification problem. 
The order of this listing corresponds to the order of numerals along the rows of 
the database.

 - R (Recency - months since last donation)
 - F (Frequency - total number of donation)
 - M (Monetary - total blood donated in c.c.)
 - T (Time - months since first donation)
 - a binary variable representing whether he/she donated blood in March 2007 (1 
stand for donating blood; 0 stands for not donating blood).

In [120]:
import pandas as pd
from sklearn.metrics import roc_auc_score
from urllib import request
import json
from pprint import pprint

In [84]:
X_test = pd.read_csv("data/webinar_9/X_test.csv")
y_test = pd.read_csv("data/webinar_9/y_test.csv")

In [85]:
X_test[['recency', 'frequency', 'time']].head()

,recency,frequency,time
0,2,7,14
1,16,3,23
2,2,5,34
3,16,14,83
4,2,7,77


In [125]:
def get_prediction(x):
    recency, frequency, _time = x
    body = {'recency': recency, 
            'frequency': frequency,
            'time': _time}
    
    myurl = "http://0.0.0.0:8180/predict"
    req = request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = request.urlopen(req, jsondataasbytes)
    return json.loads(response.read())['predictions']

In [131]:
X_test.loc[82, ['recency', 'frequency', 'time']]

recency      0
frequency    3
time         4
Name: 82, dtype: int64

In [130]:
# Проверим работоспособность функции:
get_prediction(X_test.loc[82, ['recency', 'frequency', 'time']])

0.5683573317367829

In [118]:
%%time
predictions = X_test[['recency', 'frequency', 'time']].apply(lambda x: get_prediction(x), 1)
predictions

CPU times: user 39.5 ms, sys: 4.93 ms, total: 44.5 ms
Wall time: 732 ms


0      0.567141
1      0.171273
2      0.390154
3      0.125309
4      0.209699
         ...   
102    0.314761
103    0.135539
104    0.000400
105    0.595274
106    0.100100
Length: 107, dtype: float64

In [119]:
roc_auc_score(y_score=predictions.values, y_true=y_test)

0.7844907407407408